# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4876, uptime 0:00:31
mini_beamline                    RUNNING   pid 4861, uptime 0:00:34
random_walk                      RUNNING   pid 4862, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4863, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4864, uptime 0:00:34
simple                           RUNNING   pid 4865, uptime 0:00:34
thermo_sim                       RUNNING   pid 4866, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4867, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 23:32:47
Persistent Unique Scan ID: '5c4be0e6-1d07-4198-a8a7-a65cad7fd704'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+


|         1 | 23:32:47.9 |     -1.000 |     102032 |
|         2 | 23:32:48.0 |     -0.500 |     104029 |
|         3 | 23:32:48.0 |      0.000 |     104461 |
|         4 | 23:32:48.0 |      0.500 |     104315 |
|         5 | 23:32:48.0 |      1.000 |     102871 |


+-----------+------------+------------+------------+
generator scan ['5c4be0e6'] (scan num: 3)





('5c4be0e6-1d07-4198-a8a7-a65cad7fd704',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/5c4be0e6-1d07-4198-a8a7-a65cad7fd704-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-18 23:32:48
Persistent Unique Scan ID: '70bfcfa6-4c7b-46ac-a88d-f9e29c9dd585'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 23:32:48.5 |          -1 | 1131606234 |


|         2 | 23:32:48.7 |          -0 | 1127229827 |


|         3 | 23:32:48.7 |           0 | 1122307435 |


|         4 | 23:32:48.9 |           0 | 1117618082 |


|         5 | 23:32:49.0 |           1 | 1110406364 |


+-----------+------------+-------------+------------+
generator scan ['70bfcfa6'] (scan num: 4)





('70bfcfa6-4c7b-46ac-a88d-f9e29c9dd585',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-18 23:32:49
Persistent Unique Scan ID: '55cdd287-48da-43f5-9ebc-c59289d4c9f1'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 23:32:49.6 | 1080202395 |


|         2 | 23:32:50.5 | 1055368353 |


|         3 | 23:32:51.5 | 1105637838 |


...

|        28 | 23:33:16.5 | 1128683652 |


|        29 | 23:33:17.5 | 1076801515 |


|        30 | 23:33:18.5 | 1056643410 |


+-----------+------------+------------+
generator count ['55cdd287'] (scan num: 5)





('55cdd287-48da-43f5-9ebc-c59289d4c9f1',)